In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

import plotly
import plotly.express as px
print(plotly.__version__)

import statistics
import category_encoders as ce

# тесты для отбра признаков
from sklearn.feature_selection import chi2 # хи-квадрат
from sklearn.feature_selection import f_classif # anova

#библиотеки для обработки текста
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time
nltk.downloader.download('vader_lexicon')

#библиотеки для нормализации признаков
from sklearn import preprocessing

import geopy.distance # библиотека для вычисления расстояния между точками координат

# инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

5.4.0


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/tatiana/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# 1. Подгрузка и предварительный анализ данных

In [3]:
# Подгрузим наши данные из соревнования

DATA_DIR = '/kaggle/input/sf-booking/'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн

In [ ]:
df_train.info()

In [4]:
df_train.head(1)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671


In [ ]:
df_test.info()

In [5]:
df_test.head(1)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,tags,days_since_review,lat,lng
0,Via Senigallia 6 20161 Milan Italy,904,7/21/2017,8.1,Hotel Da Vinci,United Kingdom,Would have appreciated a shop in the hotel th...,52,16670,Hotel was great clean friendly staff free bre...,62,1,"[' Leisure trip ', ' Couple ', ' Double Room '...",13 days,45.533137,9.171102


In [ ]:
sample_submission.head(2)

In [ ]:
sample_submission.info()

In [ ]:
sample_submission

In [6]:
# ВАЖНО! для корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

Всего в датасете 515738 отзывов. 
10 числовых признаков, 8 категориальных.
Пропуски в 6536 строках в признаках lat и lng. 
Всего 336 дубликатов. 
Категориальных признаков - 8, числовых - 9. 
Высокая корреляция между признаком additional_number_of_scoring и рядом других признаков.
В данных собраны отзывы о 1493 разных отеля. Есть отели с одинаковыми названиями. 

Признаки

    hotel_address - адрес отеля
    review_date - дата, когда рецензент разместил соответствующий отзыв.
    average_score - средний балл отеля, рассчитанный на основе последнего комментария за последний год
    hotel_name - название отеля
    reviewer_nationality - национальность рецензента
    negative_review - отрицательный отзыв, который рецензент дал отелю.
    review_total_negative_word_counts - общее количество слов в отрицательном отзыв
    positive_review - положительный отзыв, который рецензент дал отелю
    review_total_positive_word_counts - общее количество слов в положительном отзыве
    reviewer_score - оценка, которую рецензент поставил отелю на основе своего опыта
    total_number_of_reviews_reviewer_has_given - количество отзывов, которые рецензенты дали в прошлом
    total_number_of_reviews - общее количество действительных отзывов об отеле
    tags - теги, которые рецензент дал отелю.
    days_since_review - продолжительность между датой проверки и датой очистки
    additional_number_of_scoring - есть также некоторые гости, которые просто поставили оценку сервису, а не оставили отзыв. Это число указывает, сколько там действительных оценок без проверки.
    lat - широта отеля
    lng - долгота отеля


# 2. Очистка данных

In [ ]:
#избавляюсь от дубликатов
#data.drop_duplicates(inplace=True)

In [ ]:
data.shape[0]

In [ ]:
#ищу отели с одинаковым названиями, но разными адресами
grouped_data = data.groupby(['hotel_name','hotel_address'])['sample'].count().sort_values()

In [ ]:
grouped_data.groupby('hotel_name').count().sort_values(ascending=False)

In [ ]:
data[data['hotel_name'] == 'Hotel Regina']['hotel_address'].unique()

Три отеля, расположенные в разных городах, называются одинаково Hotel Regina. Чтобы использовать название отеля в качестве ключа, изменю названия отелей. 

In [ ]:
data.loc[(data['hotel_address'] == 'Rooseveltplatz 15 09 Alsergrund 1090 Vienna Austria'),'hotel_name'] = 'Hotel Regina in Vienna'

In [ ]:
data.loc[(data['hotel_address'] == 'Via Cesare Correnti 13 Milan City Center 20123 Milan Italy'),'hotel_name'] = 'Hotel Regina in Milan'

In [ ]:
data.shape[0]

# 3. Обработка признаков

# Признак hotel_address

In [ ]:
#выделяю страну и название города из адреса отеля
data['hotel_city'] = data['hotel_address'].apply(lambda x: x.split()[-5] if x.endswith('United Kingdom') else x.split()[-2])
data['hotel_country'] = data['hotel_address'].apply(lambda x: 'United Kingdom' if x.endswith('United Kingdom') else x.split()[-1])
data.drop('hotel_address',axis=1, inplace=True)

In [ ]:
#сводная таблица распределения городов отелей по странам
pd.pivot_table(data, values = 'sample',
               index = ['hotel_country','hotel_city'], 
               aggfunc = 'count')

Одной стране соответствует один город. Соответственно, надо будет закодировать только один из признаков.

Строю график распределения отелей по странам с учетом того, что по каждому отелю в данных существует некоторое количество отзывов.

In [ ]:
#для построения графика группирую отели по названию и стране 
country_hotels = data.groupby(['hotel_name','hotel_country'],as_index=False)['sample'].count()
country_hotels

In [ ]:
#данные для построения графика о распределении отелей по странам
country_hotels_grouped = country_hotels.groupby('hotel_country',as_index=False)['hotel_name'].count()
country_hotels_grouped.rename(columns={'hotel_name':'hotels_number'},inplace=True)
country_hotels_grouped

In [ ]:
#строю график
fig = px.bar(
    data_frame=country_hotels_grouped, 
    x='hotel_country',
    y='hotels_number', 
    color='hotel_country', 
    text = 'hotels_number', 
    orientation='v', 
    height=500, 
    width=1000, 
    title='Распределение отелей по странам' 
)

#отображаем его
fig.show()

Больше всего отелей расположены во Франции и Великобритании.

In [ ]:
#кодирую город отеля методом однократного кодирования OneHotEncoder
encoder = ce.OneHotEncoder(cols=['hotel_city']) # указываем столбец для кодирования
type_bin = encoder.fit_transform(data['hotel_city'])
data = pd.concat([data, type_bin], axis=1)

In [ ]:
#последний признак hotel_city_6 можно удалить, как неинформативный, поскольку исключение первых пяти однозначно определяет шестой город
data.drop('hotel_city_6',axis=1,inplace=True)

# Замена пустых значений признаков lat и lng на средний по городу

In [ ]:
#получаю средние значение координат отелей для каждого города
data_cleaned = data.dropna(axis=0)
data_grouped = data_cleaned.groupby(by='hotel_city',as_index=False)['lat','lng'].mean()

data_grouped.head(6)

In [ ]:
#заменяю пропущенные значения координат средними по городу
data = data.merge(data_grouped, on='hotel_city', how='left')
values = {
    'lat_x': data['lat_y'],
    'lng_x': data['lng_y']
}
data = data.fillna(values)

data.drop(['lat_y','lng_y'], axis=1, inplace=True)

# Признак расстояния от отеля до центра города

In [ ]:
# создаю словарь с координатами центров городов (координаты получены из Google Map)
city_coordinates = {'Amsterdam': (52.37016, 4.90062),
 'Barcelona': (41.37527, 2.149),
 'London': (51.50321, -0.15145),
 'Milan': (45.47261, 9.18510),
 'Paris': (48.86423, 2.33676),
 'Vienna': (48.18855, 38176)}

In [ ]:
# вычисляю расстояние до центра города
data['distance_to_center'] = data.apply(lambda x: geopy.distance.geodesic((x.lat_x, x.lng_x), city_coordinates[x.hotel_city]).m, axis=1)

In [ ]:
data['distance_to_center'].value_counts().head(10)

In [ ]:
#распределение расстояния от отелей до центра города
#выделяю уникальные значения для каждого отеля
hotels_distance = data[['hotel_name','distance_to_center']]

In [ ]:
#удаляю дубликаты, чтобы оставить уникальные строки для каждого отеля
hotels_distance = hotels_distance.drop_duplicates()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 7))
histplot = sns.histplot(data=hotels_distance, x='distance_to_center', ax=axes[0])
histplot.set_title('Распределение расстояний от отеля до центра города')
boxplot = sns.boxplot(data=hotels_distance, x='distance_to_center', ax=axes[1])
boxplot.set_title('Расстояния до центра')

In [ ]:
#использую метод MinMaxScaler
mm_scaler = preprocessing.MinMaxScaler()
hotels_distance['norm_distance_to_center'] = mm_scaler.fit_transform(hotels_distance[['distance_to_center']])

In [ ]:
#добавляю новый признак в датасет
hotels_distance.drop('distance_to_center',axis=1,inplace=True)
data = data.merge(hotels_distance, on='hotel_name',how='left')

In [ ]:
fig = plt.figure(figsize=(15, 4))
sns.histplot(data['norm_distance_to_center'], bins=60)

In [ ]:
#удаляю ненормализованный признак из датасета
data.drop('distance_to_center',axis=1,inplace=True)

In [ ]:
#удаляю признаки со значениями координат
data.drop(['lat_x','lng_x'], axis=1, inplace=True)

# Признаки страна рецензента и соответствие страны рецензента стране отеля

In [ ]:
#выясняю сколько всего национальностей рецензентов 
len(data['reviewer_nationality'].unique())


In [ ]:
#данные для построения графика
bar_data = data.groupby('reviewer_nationality',as_index=False
)[['sample']].count().nlargest(15, columns=['sample'])
bar_data.rename(columns={'sample':'reviewer_number'},inplace=True)
bar_data

In [ ]:
#строю график распределения рецензентов по странам
fig = px.bar(
    data_frame=bar_data, 
    x="reviewer_nationality", 
    y="reviewer_number", 
    color='reviewer_nationality', 
    text = 'reviewer_number', 
    orientation='v', 
    height=500, 
    width=1000, 
    title='Распределение национальности рецензентов' 
)

#отображаем его
fig.show()

Подавляющее количество рецензентов имеют гражданство Великобритании, затем США и Австралии.

In [ ]:
#обработаю признак, чтобы убрать пробелы в начале и конце строки
data['reviewer_nationality'] = data['reviewer_nationality'].apply(lambda x: x.strip())

In [ ]:
#добавляю признак соответствия национальности рецензента стране отеля
data['country_nationality_accordance'] = (data['hotel_country'] == data['reviewer_nationality']).astype(int)

In [ ]:
#оставлю 10 наиболее часто встречающихся национальностей. Остальные поменяю на Other
top_ten = data['reviewer_nationality'].value_counts().nlargest(10).index
data['reviewer_nationality'] = data['reviewer_nationality'].apply(lambda x: x if x in top_ten else 'Other')

In [ ]:
data['reviewer_nationality'].unique()

In [ ]:
#кодирую признак национальности рецензента методом двоичного кодирования
bin_encoder = ce.BinaryEncoder(cols=['reviewer_nationality']) # указываем столбец для кодирования
type_bin = bin_encoder.fit_transform(data['reviewer_nationality'])
data = pd.concat([data, type_bin], axis=1)


In [ ]:
data.drop(['hotel_city','hotel_country','reviewer_nationality'],axis=1,inplace=True)

# Признак даты отзыва

In [ ]:
#выделяю месяц, когда был написан отзыв
data['review_date'] = pd.to_datetime(data['review_date'])
data['review_month'] = data['review_date'].dt.month

In [ ]:
#дата самого раннего отзыва
data['review_date'].min()

In [ ]:
#дата последнего отзыва
data['review_date'].max()

In [ ]:
#кодирую признак в высокий или низкий сезон состоялось посещение отеля
data['high_season'] = data['review_month'].apply(lambda x: 1 if x in [1,5,6,7,8,9] else 0)

# Признак срока давности отзыва

In [ ]:
#вычисляю признак давности отзыва в месяцах относительно даты последнего отзыва
from datetime import datetime
data['age_of_review'] = data['review_date'] - data['review_date'].min()
data['age_of_review'] = data['age_of_review'].astype(str)
data['age_of_review'] = data['age_of_review'].apply(lambda x: x.split()[0])
data['age_of_review'] = round(data['age_of_review'].astype(int) / 30)

In [ ]:
data['age_of_review'].min()

In [ ]:
fig = plt.figure(figsize=(15, 4))
sns.histplot(data['age_of_review'], bins=60)

In [ ]:
data.drop(['review_date','review_month'],axis=1,inplace=True)

In [ ]:
#попробую применить RobustScaler
r_scaler = preprocessing.RobustScaler()
data['age_of_review'] = r_scaler.fit_transform(data[['age_of_review']])

# Признак Отрицательный отзыв

In [ ]:
#обрабатываю признак, чтобы убрать пробелы в начале и конце строки
data['negative_review'] = data['negative_review'].apply(lambda x: x.strip())

In [ ]:
#проверяю, какие негативные отзывы наиболее часто втречаются
data['negative_review'].value_counts().nlargest(10)

In [ ]:
#список ложных отрицательных отзывов 
false_negative = ['We loved everything','not much','Leaving','Was all good','','Coming home','Noting','absolutely nothing','nothing all good','Nothing all great','Nothing It was perfect','Can t think of s thing','Nothing everything was great','No problems at all','All was great','Nothing All good','Not a lot','I liked it all','Cannot think of anything','Nothing negative to say','No comments','Liked it all','Going home','All great','Nothing could have been better','Nothing bad to say','Everything was excellent','No dislikes','Nothing to say','There was nothing we didn t like','All ok','no complaints','No complaints at all','There was nothing to dislike','Nothing I didn t like','As above','nothing really','It was all good','Nothing not to like','Having to leave','Nothing all good','Nothing could have been improved','Can t think of anything really','No Negative','Nothing','nothing','None','N A',' ','Nothing really','N a','All good','No complaints',
                  'Nothing at all','Nothing to dislike','none','Nil','Everything was perfect','Can t think of anything',
                 'n a','Absolutely nothing','Everything was great','Nothing to complain about','NA','NOTHING','I liked everything',
                 'No','Liked everything','Not much','Everything was good','See above','Nothing in particular','Everything was fine','Na',
                 'All was good','Nothing it was perfect','Not applicable','Non','Nothing comes to mind','There was nothing I didn t like',
                 'We liked everything','Nothing everything was perfect','No negatives','Nothing everything was great','Nothing to report',
                 'Not a thing','Loved everything','No thing','all good','There was nothing not to like','No issues','Nothing I can think of',
                 'No bad experience','Nothing to mention','no','No comment','Nothing we didn t like','Nothing to complain','No problems',
                 'I loved everything','nil','nothing at all','nothing bad','Couldn t fault it','Nothing to fault','It was perfect',
                 'Nothing Everything was perfect','There wasn t anything we didn t like','nothing to dislike','Nothing much','non',
                 'That we couldn t stay longer','That we had to leave','No bad experiences','Loved it all','That I couldn t stay longer',
                 'I have no complaints','Nothing it was all good','I can t think of anything','Nothing that I can think of','Nothing of note',
                 'Couldn t fault anything','nothing to complaint about','All was perfect','I had to leave','That I had to leave','Very little','leaving',
                 'everything was perfect','There wasn t anything I didn t like','Having to go home','Nithing','Nothing all was great',
                 'Everything was very good','No complaints whatsoever','Everything was fantastic','Could not fault anything','No complains',
                 'Can t think of any','Nope','Nothing specific','everything was great','no thing','Everything perfect','Can t fault it',
                 'Happy with everything','No one thing','Everything good','Nothing everything was good','There is nothing to dislike',
                 'Nothing I did not like','Nothing to note','Nothings','Nothing all perfect','Nothing bad to report','I','Nothing was bad',
                 'all ok','Nothing to complain about at all','Nothing that I didn t like','Nothinh','We had to leave','Nothing everything was excellent',
                 'There wasn t anything to not to like', 'Can t think of anything I didn t like','I loved it all','Having to come home',
                 'Nothing bad at all','nothing in particular','All very good','nothing comes to mind','There was nothing','Everything was just perfect',
                 'Nothing to say here','Nothing all excellent','I don t know','No faults at all','Wish we could stay longer','Nothing loved it all',
                 'Norhing','Nothing All was great','nothing to say','There wasn t anything','Almost nothing','No things','Nothing that we didn t like',
                 'Nothing to comment','All OK','Not much really','Nothing as such','Nothing to improve','Honestly nothing','Nowt','Nothing to add',
                 'My stay was too short','Nothing it was amazing','I like everything','Nothing so far','nothing to mention','All is good',
                 'Had to leave','nope']

In [ ]:
#создаю признак наличия либо отсутствия негативного отзыва
data['negative_review_exists'] = data['negative_review'].apply(lambda x: 0 if x in false_negative else 1 )

# Признак Процент истиных негативных отзывов по каждому отелю

In [ ]:
#групирую данные
grouped = data.groupby(['hotel_name','total_number_of_reviews'],as_index=False)['negative_review_exists'].sum()
grouped.head(2)

In [ ]:
#считаю процент истиных негативных отзывов по каждому отелю
grouped['%_of_negative_reviews'] = grouped['negative_review_exists'] / grouped['total_number_of_reviews']

In [ ]:
grouped['%_of_negative_reviews'].hist()

In [ ]:
grouped['%_of_negative_reviews'].median()

In [ ]:
#добавляю признак в данные
grouped.drop(['total_number_of_reviews','negative_review_exists'],axis=1,inplace=True)
data = data.merge(grouped, on='hotel_name',how='left')

In [ ]:
data['%_of_negative_reviews'].hist()

In [ ]:
data['%_of_negative_reviews'].median()

# Признак Положительный отзыв

In [ ]:
#обработаю признак, чтобы убрать пробелы в начале и конце строки
data['positive_review'] = data['positive_review'].apply(lambda x: x.strip())

In [ ]:
#проверяю, какие позитивные отзывы наиболее часто втречаются
data['positive_review'].value_counts().nlargest(10)

In [ ]:
#список ложных положительных отзывов
false_positive = ['No Positive','Nothing','nothing','Not much','']

In [ ]:
#создаю признак наличия либо отсутствия истиного положительного отзыва
data['positive_review_exists'] = data['positive_review'].apply(lambda x: 0 if x in false_positive else 1 )

# Признак Процент истиных положительных отзывов

In [ ]:
grouped = data.groupby(['hotel_name','total_number_of_reviews'],as_index=False)['positive_review_exists'].sum()
grouped.head(2)

In [ ]:
grouped['%_of_positive_reviews'] = grouped['positive_review_exists'] / grouped['total_number_of_reviews']

In [ ]:
grouped['%_of_positive_reviews'].hist()

In [ ]:
#добавляю новый признак в датасет
grouped.drop(['total_number_of_reviews','positive_review_exists'],axis=1,inplace=True)
data = data.merge(grouped, on='hotel_name',how='left')

In [ ]:
data['%_of_positive_reviews'].hist()

In [ ]:
data.drop(['total_number_of_reviews'],axis=1,inplace=True)

# Признаки процентного содержания негативных и позитивных слов в отзыве

In [ ]:
#поскольку количество слов в негативных и позитивных отзывах, указанное в признаках не соответствует действительности, пересчитаю эти значения
data['review_total_negative_word_counts'] = data['negative_review'].apply(lambda x: len(x.split()))
data['review_total_positive_word_counts'] = data['positive_review'].apply(lambda x: len(x.split()))

In [ ]:
#заменяю значение количества слов в ложных негативных отзывах на 0
data.loc[(data['negative_review_exists'] == 0), 'review_total_negative_word_counts'] = 0
data.loc[(data['positive_review_exists'] == 0), 'review_total_positive_word_counts'] = 0

In [ ]:
data[['negative_review','review_total_negative_word_counts','positive_review','review_total_positive_word_counts']].head(6)

In [ ]:
data['review_total_negative_word_counts'].max()

In [ ]:
#проверяю, что отзывы длинной в несколько сотен слов это реальные отзывы, а не случайные выбросы
data[data['review_total_negative_word_counts'] > 300]['negative_review'].unique()[0]

In [ ]:
#вычисляю процент содержания негативных и позитивных слов в отзыве
data['%_of_negative_words'] = data['review_total_negative_word_counts'] / (data['review_total_negative_word_counts'] + data['review_total_positive_word_counts'])
data['%_of_positive_words'] = data['review_total_positive_word_counts'] / (data['review_total_negative_word_counts'] + data['review_total_positive_word_counts'])

In [ ]:
#проверяю признаки на наличие пропусков
cols_null_percent = data[['%_of_negative_words','%_of_positive_words']].isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
display(cols_with_null)

In [ ]:
#заменяю пропуски на 0
data['%_of_negative_words'] = data['%_of_negative_words'].fillna(0)
data['%_of_positive_words'] = data['%_of_positive_words'].fillna(0)

In [ ]:
#проверяю признаки на наличие np.inf значений
np.all(np.isfinite(data[['%_of_negative_words','%_of_positive_words']]))

In [ ]:
data[['%_of_negative_words','%_of_positive_words']].hist()


# Признак разности количества слов в положительном и отрицательном отзывах

In [ ]:
data['dif_of_positive_negative'] = (data['review_total_positive_word_counts'] + 1) / (data['review_total_negative_word_counts'] + 1) / 100

In [ ]:
data['dif_of_positive_negative'].hist()

# Признаки удовлетворенности рецензента отелем

In [ ]:
# пакет символьной и статистической обработки текста для анализа отзывов
sent_analyzer = SentimentIntensityAnalyzer()

# анализ отзывов
data['negative_sent_review'] = data['negative_review'].apply(lambda x: sent_analyzer.polarity_scores(x))
data['positive_sent_review'] = data['positive_review'].apply(lambda x: sent_analyzer.polarity_scores(x))

# записываю результаты в основной датафрейм в виде отдельных признаков
data.loc[:,['n_neg', 'n_neu', 'n_pos', 'n_compound']] = list(data['negative_sent_review'].apply(lambda x: [x['neg'], x['neu'], x['pos'], x['compound']]).values)
data.loc[:,['p_neg', 'p_neu', 'p_pos', 'p_compound']] = list(data['positive_sent_review'].apply(lambda x: [x['neg'], x['neu'], x['pos'], x['compound']]).values)
data.head(3)

In [ ]:
data.drop(['negative_review','positive_review','negative_sent_review','positive_sent_review'],axis=1,inplace=True)

# Признак tags

In [ ]:
data['tags'].unique()

Закодирую 30 наиболее часто встречающихся тегов и обработаю получившиеся признаки.

In [ ]:
#избавляюсь от прямых скобок
data['tags'] = data['tags'].apply(lambda x: x.strip('[]'))

In [ ]:
#избавляюсь от кавычек и пробелов
data['tags'] = data['tags'].apply(lambda x: [w.strip(" '") for w in x.split(',')])

In [ ]:
display(data['tags'])

In [ ]:
#считаю количество одинаковых тегов
tags_dict = {}
def fill_tags_dict(x):
    for elem in x:
        if elem in tags_dict:
            tags_dict[elem] = tags_dict[elem] + 1
        else:
            tags_dict[elem] = 1
    return x
data['tags'].apply(fill_tags_dict) 

In [ ]:
len(tags_dict)

In [ ]:
#создаю DataFrame, отсортированный по частоте втречаемости тегов
tags_count = pd.DataFrame(tags_dict,index=['count']).transpose()
tags_sorted = tags_count.sort_values(by='count', ascending=False).head(30)

In [ ]:
display(tags_sorted)

In [ ]:
#создаю список наиболее часто встречающихся тегов
tags_30 = list(tags_sorted.index)

In [ ]:
display(tags_30)

In [ ]:
#кодирую наиболее часто встречающиеся теги
for tag in tags_30:
    data[tag] = data['tags'].apply(lambda x: 1 if tag in x else 0)

In [ ]:
#предположу, что продолжительность проживания в отеле в течении одного или двух дней может повлиять на характер оценки отеля
#удаляю ненужные признаки
data.drop(['Stayed 3 nights','Stayed 4 nights','Stayed 5 nights','Stayed 6 nights','Stayed 7 nights'], axis=1, inplace=True)

In [ ]:
#объединяю теги для номеров одинаковой категории
#standard double or twin room
data['standard_double_twin_room'] = ((data['Double Room']==1)|(data['Standard Double Room']==1)|(data['Double or Twin Room']==1)|
                        (data['Standard Double or Twin Room']==1)|(data['Classic Double Room']==1)|(data['2 rooms']==1)|
                        (data['Standard Twin Room']==1)|(data['Twin Room']==1)|(data['Classic Double or Twin Room']==1)).astype(int)
data.drop(['Double Room','Standard Double Room','Double or Twin Room','Standard Double or Twin Room','Classic Double Room','2 rooms',
          'Standard Twin Room','Twin Room','Classic Double or Twin Room'],axis=1,inplace=True)
#superior double or twin room
data['superior_double_twin_room'] = ((data['Superior Double Room']==1)|(data['Superior Double or Twin Room'])|
                        (data['Superior Twin Room'])|(data['Deluxe Double Room'])).astype(int)
data.drop(['Superior Double Room','Superior Double or Twin Room','Superior Twin Room','Deluxe Double Room'],axis=1,inplace=True)

In [ ]:
data.drop(['tags'], axis=1, inplace=True)

In [ ]:
data_fixed = data.copy()

In [ ]:
#data = data_fixed

# 5. Подготовка данных к обучению модели

In [ ]:
# убираем признаки которые еще не успели обработать, 
# модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
object_columns = [s for s in data.columns if data[s].dtypes == 'object']
data.drop(object_columns, axis = 1, inplace=True)

In [ ]:
#проверяю признаки на мультиколлинеарность
plt.rcParams['figure.figsize'] = (30,30)
sns.heatmap(data.drop(['sample'], axis=1).corr(), annot=True,cmap='coolwarm')

In [ ]:
#убираю сильно коррелирующие признаки на основании матрицы
data.drop(['Business trip','%_of_negative_reviews','positive_review_exists','negative_review_exists','n_compound','p_pos','p_neu','n_compound','review_total_negative_word_counts','%_of_positive_words','hotel_city_4'],axis=1,inplace=True)

In [ ]:
#убираю неинформативные признаки
data.drop(['additional_number_of_scoring','total_number_of_reviews_reviewer_has_given'],axis=1,inplace=True)

In [ ]:
#стандартизирую признаки
col_name= data[['average_score',
                'review_total_positive_word_counts'
               ]]
scaler = preprocessing.StandardScaler()
data_scalar = scaler.fit_transform(col_name)
data_scalar = pd.DataFrame(data_scalar,columns=col_name.columns)

Проверяю признаки на значимость

In [ ]:
data.columns

In [ ]:
# непрерывные признаки
num_cols = ['average_score','review_total_positive_word_counts','norm_distance_to_center',
            'age_of_review','%_of_positive_reviews','%_of_negative_words',
            'dif_of_positive_negative','n_neg','n_neu','n_pos','p_neg','p_compound']

# категориальные признаки
cat_cols = ['average_score','country_nationality_accordance', 'high_season', 'Leisure trip','Submitted from a mobile device',
            'Couple', 'Stayed 1 night', 'Stayed 2 nights','Group','Family with young children',
            'Family with older children','Single Room','Executive Double Room','standard_double_twin_room',
            'superior_double_twin_room','hotel_city_1','hotel_city_2','hotel_city_3',
        'reviewer_nationality_0','reviewer_nationality_1','reviewer_nationality_2','reviewer_nationality_3']
           

In [ ]:
data[num_cols].describe()

In [ ]:
y = data.query('sample == 1').drop(['sample'], axis=1).reviewer_score.values.astype('int')
X = data.query('sample == 1').drop(['sample'], axis=1)[cat_cols]

plt.rcParams['figure.figsize'] = (15,10)
imp_cat = pd.Series(chi2(X, y)[0], index=cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

In [ ]:
y = data.query('sample == 1').drop(['sample'], axis=1).reviewer_score.values.astype('int')
X = data.query('sample == 1').drop(['sample'], axis=1)[num_cols]

imp_num = pd.Series(f_classif(X, y)[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

In [ ]:
# Визуализация пропусков
plt.figure(figsize=(6,4))
sns.heatmap(data.isna().transpose(),
            cmap="YlGnBu",
            cbar_kws={'label': 'Пропущенные данные'})
plt.show()

In [ ]:
#проверяю данные на наличие значение np.inf
np.all(np.isfinite(data))

# 6. Обучение модели и анализ результатов

In [ ]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# функция Mean Absolute Percentage Error (MAPE)
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))
print('MAPE:', mean_absolute_percentage_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['reviewer_score'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
list(sample_submission)

In [ ]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)